# 휴먼지능정보공학과 201810794 이은후

In [24]:
%%writefile src/CustomerOrder.sol
pragma solidity 0.5.1;
contract Customer {
    struct CustomerInfo {
        uint id;
        string name;
        string phoneNum;
        string destination;
        bool isCustomer; //false
    }
    mapping (address => CustomerInfo) customerMap;
    mapping (address => uint) mileage;
    function addCustomer(address addr, uint _id, string memory _name, string memory _phoneNum, string memory _dest) public {
        if (customerMap[addr].isCustomer == false) {
            CustomerInfo memory c = CustomerInfo(_id, _name, _phoneNum, _dest, true);
            customerMap[addr] = c;
        }
    }
    function getCustomerDestination(address addr) view public returns (string memory) {
        CustomerInfo memory c = customerMap[addr];
        return c.destination;
    }
    function setMileage(address addr, uint amount) public {
        mileage[addr] = amount / 100;
    }
}

contract Order {
    Customer customer;
    constructor() public { 
        customer = new Customer();
    }
    struct OrderInfo {
        uint oid;
        string oname;
        uint256 timestamp;
    }
    mapping (address => OrderInfo) orderMap;
    mapping (uint => address) addressById;
    mapping (address => string) orderStatus;
    mapping (address => uint) orderQuantity;
    mapping (address => uint) orderAmount;
    mapping (address => bool) isOrdered;
    uint accAmount;
    uint accOrderNum;
    function ordering(address payable addr, uint _oid, string memory _oname, uint _quantity, uint _amount) public payable {
        OrderInfo memory o = OrderInfo(_oid, _oname, now);
        orderMap[addr] = o;
        addressById[_oid] = addr;
        orderStatus[addr] = "Ordered";
        orderQuantity[addr] =_quantity;
        orderAmount[addr] = _amount;
        isOrdered[addr] = true;
        accAmount += _amount;
        accOrderNum ++;
        customer.setMileage(addr, _amount);
    }
    function addCustomer(address addr, uint _id, string memory _name, string memory _phoneNum, string memory _dest) public {
        customer.addCustomer(addr, _id, _name, _phoneNum, _dest);
    }
    function getCustomerDestination(address _addr) public view returns(string memory) {
        return customer.getCustomerDestination(_addr);
    }
    function getOrderByAddress(address _addr) view public returns (uint, string memory, string memory, string memory) {
        OrderInfo memory o = orderMap[_addr];
        return (o.oid, o.oname, orderStatus[_addr], customer.getCustomerDestination(_addr));
    }
    function getOrderById(uint _id) view public returns (uint, string memory, string memory, string memory) {
        OrderInfo memory order = orderMap[addressById[_id]];
        return (order.oid, order.oname, orderStatus[addressById[_id]], customer.getCustomerDestination(addressById[_id]));
    }
    function getInfo() public view returns(uint, uint, uint) {
        return (accOrderNum, accAmount, address(this).balance);
    }
    function refundById(uint _id) public payable{
        OrderInfo memory order = orderMap[addressById[_id]];
        require(_id == order.oid);
        accAmount -= orderAmount[addressById[_id]];
        orderStatus[addressById[_id]] = "Refunded";
        orderAmount[addressById[_id]] = 0;
        accOrderNum --;
    }
    function refundResult(uint _id) public view returns (uint, string memory, uint, string memory, string memory) {
        OrderInfo memory o = orderMap[addressById[_id]];
        return (o.oid, o.oname, orderAmount[addressById[_id]], orderStatus[addressById[_id]], customer.getCustomerDestination(addressById[_id]));
    }
}

Overwriting src/CustomerOrder.sol


In [25]:
!solc src/CustomerOrder.sol --combined-json abi,bin > src/CustomerOrder.json

In [26]:
%%writefile src/CustomerOrderDeploy.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts["src/CustomerOrder.sol:Order"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 3000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/CustomerOrderDeploy.js


In [27]:
!node src/CustomerOrderDeploy.js

Deploying the contract from 0x41a78c873d7086587E391d0B352C6248F7EFfA7c
>>> transactionHash0xbdc31818fcc5729f59e2f78b3423ebd1602eaad956ec567f2c61dea6bba2ee5a
>>> RECEPIT hash: 0xbdc31818fcc5729f59e2f78b3423ebd1602eaad956ec567f2c61dea6bba2ee5a
>>> address:0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E
---> The contract deployed to: 0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E


In [28]:
%%writefile src/CustomerOrderUse_1.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);

var order = new web3.eth.Contract(_abiArray, "0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account1: " + accounts[0]);
    const balanceAcc1 = await web3.eth.getBalance(accounts[0]);
    console.log("Account1 balance: " + balanceAcc1);
    console.log("\n");
    console.log("Account2: " + accounts[1]);
    const balanceAcc2 = await web3.eth.getBalance(accounts[1]);
    console.log("Account2 balance: " + balanceAcc2);
    console.log("\n");
    console.log("Account3: " + accounts[2]);
    const balanceAcc3 = await web3.eth.getBalance(accounts[2]);
    console.log("Account3 balance: " + balanceAcc3);
    console.log("\n");
    //const blockNum = await web3.eth.blockNumber
    console.log("Block number: " + web3.eth.getBlockNumber().then(console.log));
}

doIt()

Overwriting src/CustomerOrderUse_1.js


## 답: 1-1 주소 및 잔고 출력

In [29]:
!node src/CustomerOrderUse_1.js

Account1: 0x41a78c873d7086587E391d0B352C6248F7EFfA7c
Account1 balance: 99898643059999998885


Account2: 0x7DC55B191230385b7ec5dc30C9eBBC12ACAb8503
Account2 balance: 99995551939999998884


Account3: 0x8fDf446A8f49E3D9fefBe06A3AF33c6b88E39C0B
Account3 balance: 99995551939999998883


Block number: [object Promise]
11


## 답: 1-2 ```Order``` 컨트랙 생성에 필요한 gas 출력
string 사용으로 인해 Order컨트랙의 gas가 infinite로 출력되고 있음.

In [30]:
!solc --gas src/CustomerOrder.sol


======= src/CustomerOrder.sol:Customer =======
Gas estimation:
construction:
   480 + 447200 = 447680
external:
   addCustomer(address,uint256,string,string,string):	infinite
   getCustomerDestination(address):	infinite
   setMileage(address,uint256):	20403

======= src/CustomerOrder.sol:Order =======
Gas estimation:
construction:
   infinite + 1685800 = infinite
external:
   addCustomer(address,uint256,string,string,string):	infinite
   getCustomerDestination(address):	infinite
   getInfo():	1157
   getOrderByAddress(address):	infinite
   getOrderById(uint256):	infinite
   ordering(address,uint256,string,uint256,uint256):	infinite
   refundById(uint256):	infinite
   refundResult(uint256):	infinite


In [31]:
%%writefile src/CustomerOrderUse_3.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);

var order = new web3.eth.Contract(_abiArray, "0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    await order.methods.addCustomer(accounts[0], 111, "kim", "010-2017-1111", "111 hongji-dong jongro-gu seoul").send({from:accounts[3],gas:5000000});
    await order.methods.addCustomer(accounts[1], 112, "lee", "010-2017-1112", "112 hongji-dong jongro-gu seoul").send({from:accounts[3],gas:5000000});
    await order.methods.addCustomer(accounts[2], 113, "lim", "010-2017-1113", "113 hongji-dong jongro-gu seoul").send({from:accounts[3],gas:5000000});
}

doIt()

Overwriting src/CustomerOrderUse_3.js


## 답: 1-3 고객정보 3건 입력

In [32]:
!node src/CustomerOrderUse_3.js

In [33]:
%%writefile src/CustomerOrderUse_4.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);

var order = new web3.eth.Contract(_abiArray, "0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    order.methods.getCustomerDestination(accounts[0]).call().then(function(value){console.log(value);});
    order.methods.getCustomerDestination(accounts[1]).call().then(function(value){console.log(value);});
    order.methods.getCustomerDestination(accounts[2]).call().then(function(value){console.log(value);});
}

doIt()

Overwriting src/CustomerOrderUse_4.js


## 답: 1-4 모든 고객의 배송지 출력

In [34]:
!node src/CustomerOrderUse_4.js

111 hongji-dong jongro-gu seoul
112 hongji-dong jongro-gu seoul
113 hongji-dong jongro-gu seoul


In [35]:
%%writefile src/CustomerOrderUse_56.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);

var order = new web3.eth.Contract(_abiArray, "0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    await order.methods.ordering(accounts[0], 555, "T-Shirt", 2, 1115).send({from:accounts[0],value:1115,gas:5000000});
    await order.methods.ordering(accounts[1], 556, "T-Shirt", 3, 1116).send({from:accounts[1],value:1116,gas:5000000});
    await order.methods.ordering(accounts[2], 557, "T-Shirt", 4, 1117).send({from:accounts[2],value:1117,gas:5000000});
    order.methods.getInfo().call().then(function(value){console.log(value);});
}

doIt()

Overwriting src/CustomerOrderUse_56.js


## 답: 1-5 주문 & 1-6 주문개수, 주문금액 합계 및 잔고 출력
문제 1-5에 해당하는 주문을 한 후, 1-6에 해당하는 주문개수, 주문금액 합계, 컨트랙잔고 출력

In [36]:
!node src/CustomerOrderUse_56.js

Result { '0': '3', '1': '3348', '2': '3348' }


In [37]:
%%writefile src/CustomerOrderUse_7.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);

var order = new web3.eth.Contract(_abiArray, "0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    order.methods.getOrderByAddress(accounts[0]).call().then(function(value){console.log(value);});
    order.methods.getOrderByAddress(accounts[1]).call().then(function(value){console.log(value);});
    order.methods.getOrderByAddress(accounts[2]).call().then(function(value){console.log(value);});
}

doIt()

Overwriting src/CustomerOrderUse_7.js


## 답: 1-7 모든 고객의 주문 내역 출력

In [38]:
!node src/CustomerOrderUse_7.js

Result {
  '0': '555',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '111 hongji-dong jongro-gu seoul'
}
Result {
  '0': '556',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '112 hongji-dong jongro-gu seoul'
}
Result {
  '0': '557',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '113 hongji-dong jongro-gu seoul'
}


In [39]:
%%writefile src/CustomerOrderUse_8.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);

var order = new web3.eth.Contract(_abiArray, "0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    order.methods.getOrderById(556).call().then(function(value){console.log(value);});
}

doIt()

Overwriting src/CustomerOrderUse_8.js


## 답: 1-8 주문ID 556으로 주문내역 출력

In [40]:
!node src/CustomerOrderUse_8.js

Result {
  '0': '556',
  '1': 'T-Shirt',
  '2': 'Ordered',
  '3': '112 hongji-dong jongro-gu seoul'
}


In [41]:
%%writefile src/CustomerOrderUse_9.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);

var order = new web3.eth.Contract(_abiArray, "0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    await order.methods.refundById(556).send({from:accounts[3],to:accounts[1],value:1116,gas:5000000});
    const balanceAcc2 = await web3.eth.getBalance(accounts[1]);
    console.log("Account2 balance: " + balanceAcc2);
}

doIt()

Overwriting src/CustomerOrderUse_9.js


## 답: 1-9 주문ID 556으로 환불처리
환불처리를 한 후, 주소2의 잔고변동 출력

In [42]:
!node src/CustomerOrderUse_9.js

Account2 balance: 99991103879999997768


In [43]:
%%writefile src/CustomerOrderUse_10.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/CustomerOrder.json");
var _json=JSON.parse(_str)
var _abiArray=JSON.parse(_json.contracts["src/CustomerOrder.sol:Order"].abi);

var order = new web3.eth.Contract(_abiArray, "0xED83FADa17a7E6820eD678aae13Ef8296Ceae41E");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    order.methods.refundResult(556).call().then(function(value){console.log(value);});
    order.methods.getInfo().call().then(function(value){console.log(value);});
}

doIt()

Writing src/CustomerOrderUse_10.js


## 답: 1-10 환불처리가 반영된 출력

In [44]:
!node src/CustomerOrderUse_10.js

Result {
  '0': '556',
  '1': 'T-Shirt',
  '2': '0',
  '3': 'Refunded',
  '4': '112 hongji-dong jongro-gu seoul'
}
Result { '0': '2', '1': '2232', '2': '4464' }
